# 计算机视觉项目开发-实训

## 实训的综合安排
**实训目的**
1. 掌握网上平台的使用方法，能进行AI项目的开发。
1. 掌握计算机视觉项目开发的基本流程。
1. 巩固、扩展使用TensorFlow框架等专业能力。

**实训内容：**

1. 完成网上平台的注册，熟悉平台的基本使用方法。
1. 参照案例，在网上平台实现计算机视觉项目。
1. 实现采用不同模型进行识别，并比较它们的效果。
1. 能够存储和读取模型。
1. 完成相关文档。

**主要参考资料：**
1. kaggle上本文案例
1. 百度等搜索工具
1. bilibili等视频网站
1. kaggle网上其它相关案例


## 花卉识别案例

### 准备工作

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import json
import matplotlib.pyplot as plt
import cv2

In [2]:
tf.__version__

'2.11.0'

## 数据准备

In [4]:
img_height = 224
img_width = 224

In [5]:
#建立数据集Dataset，数据集的用法见百度，例如 https://blog.csdn.net/qq_36201400/article/details/108827669
train_ds = tf.keras.utils.image_dataset_from_directory(
  '../input/pytorch-challange-flower-dataset/dataset/train',
  validation_split=None,
  image_size=(img_height, img_width),
  batch_size=32,
  seed=42,
  shuffle=True)

Found 6552 files belonging to 102 classes.


In [6]:
with open('../input/pytorch-challange-flower-dataset/cat_to_name.json') as f:
    cat_to_name = json.load(f)


In [7]:
type(train_ds)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [8]:
#检测每一个元素中的component的类型
train_ds.element_spec

(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))

In [15]:
type(data_test)

tuple

In [26]:
data_test =train_ds.as_numpy_iterator().next()
print('images shape: ',data_test[0].shape)
print('labels shape: ',data_test[1].shape)

images shape:  (32, 224, 224, 3)
labels shape:  (32,)


In [ ]:
plt.figure(figsize=(12,8))
for idx in range(12):
    plt.subplot(3,4,idx+1)
    plt.imshow(data_test[0][idx].astype('uint8'))
    _cat_id = str(data_test[1][idx])
    _title = _cat_id+': '+cat_to_name[_cat_id]
    plt.title(_title)
    plt.show_grid('off')    

In [25]:
#查看花卉名称,注意，输入的是字符串
cat_to_name['1']

'pink primrose'

In [ ]:
data_tuple[0].max()

In [ ]:
# import tensorflow_datasets as tfds
plt.imshow(data_tuple[0][2].astype('uint8'))   #作为浮点数输出的话，范围砸[0.0.1.0]

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  '../input/pytorch-challange-flower-dataset/dataset/valid',
  validation_split=None,
  image_size=(img_height, img_width),
  batch_size=32,
  seed=42,
  shuffle=True)

In [ ]:
def flower_name(val):
    return cat_to_name[train_ds.class_names[val]]

In [ ]:
# train_ds.class_names
train_ds.class_names[1]

In [ ]:
flower_name(1)

In [ ]:
def to_internal_tag(name_tag):
    return train_ds.class_names.index(str(name_tag))

In [ ]:
base_model = tf.keras.applications.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(img_height, img_width, 3)
)
base_model.trainable = False

### 建立模型

In [ ]:
# the model: using VGG16 as a base model and apply transfer learning on it

inputs = tf.keras.Input(shape=(img_height, img_width, 3))
x = tf.keras.applications.vgg16.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(102)(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### 训练模型

In [ ]:
num_epochs = 20
history = model.fit(train_ds, validation_data=val_ds, epochs=num_epochs,
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            # Stop training when `val_loss` is no longer improving
            monitor="val_loss",
            # "no longer improving" being defined as "no better than 1e-2 less"
            min_delta=1e-2,
            # "no longer improving" being further defined as "for at least 2 epochs"
            patience=2,
            verbose=1,
        )
    ]
)

## 观察分析模型

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(num_epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import random
import os
img_name = random.choice(os.listdir('../input/pytorch-challange-flower-dataset/dataset/test/'))
img_path = '../input/pytorch-challange-flower-dataset/dataset/test/' + img_name

img = tf.keras.utils.load_img(
    img_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "{} most likely belongs to {} with a {:.2f} percent confidence."
    .format(img_name, flower_name(np.argmax(score)), 100 * np.max(score))
)

plt.figure(figsize=(2, 2))
plt.imshow((img_array[0].numpy()).astype('uint8'))
plt.title("{}:{:.2f}".format(flower_name(np.argmax(score)), 100 * np.max(score)))
plt.axis('off')

In [ ]:
#Here we use data from another larger dataset with the same labels for testing accuracy.
test_x = [cv2.resize(plt.imread('../input/flower-classification-dataset/files/0.jpg'), dsize=(img_height, img_width))]
